In [1]:
import pandas as pd
import numpy as np
import datetime
import math
Billing = pd.read_csv("FinalCSVs/Billing.csv")
DiseaseCase = pd.read_csv("FinalCSVs/DiseaseCase.csv")

/Users/davidhuang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# Compare # of unique DM2 patients in Billing and DiseaseCase
#diabetes = ["Diabetes Mellitus (ml)","Diabetes Mellitus"]
DiseaseCase['Disease'] = DiseaseCase['Disease'].replace("Diabetes Mellitus (ml)","Diabetes Mellitus") # Make all DM2 diagnosis' the same
DC_dm2 = DiseaseCase.loc[DiseaseCase['Disease'] == "Diabetes Mellitus"]
DC_dm2_id = DC_dm2['Patient_ID'].unique()
print(len(DC_dm2_id))

20916


In [3]:
Billing2 = Billing.dropna(subset=['DiagnosisCode_calc']) # Remove NA
Billing2 = Billing2[pd.to_numeric(Billing2.DiagnosisCode_calc,errors='coerce').notnull()] # convert diagnosis codes to floats
Billing2['DiagnosisCode_calc'] = pd.to_numeric(Billing2['DiagnosisCode_calc']) # convert diagnosis codes to floats
BL_dm2 = Billing2.loc[(Billing2['DiagnosisCode_calc'] >= 250.0) & (Billing2['DiagnosisCode_calc'] < 251.0)] #Extract DM2 diagnosis patients into new dataframe
BL_dm2_id = BL_dm2['Patient_ID'].unique()
print(len(BL_dm2_id)) #print number of unique patients w/ DM2
# The unique list of DM2 patients_IDs are stored in the DC_dm2_id variable

3380


In [4]:
# Check intersection between BL and DC DM2 patients
np.intersect1d(DC_dm2_id, BL_dm2_id).size # Therefore some DM2 diagnosis in BL not in DC

3362

In [5]:
# Concatenate DM2 patients from both sheets
DC_dm2_c = DC_dm2[['Patient_ID', 'Disease','DateOfOnset']]
BL_dm2_c = BL_dm2[['Patient_ID','ServiceDate','DiagnosisCode_calc']]
BL_dm2_c.columns = ['Patient_ID','DateOfOnset','DiagnosisCode_calc']
BL_dm2_c['Disease'] = "Diabetes Mellitus"

DM2_diags = pd.concat([DC_dm2_c,BL_dm2_c], axis=0, ignore_index=True)
print(len(DC_dm2_c.index),len(BL_dm2_c.index),len(DM2_diags.index))
print(DM2_diags.head())

39457 9482 48939
  DateOfOnset  DiagnosisCode_calc            Disease   Patient_ID
0  2009-04-29                 NaN  Diabetes Mellitus  80013224228
1  2019-01-18                 NaN  Diabetes Mellitus  80034696074
2  2015-04-23                 NaN  Diabetes Mellitus  80080116812
3  2011-11-01                 NaN  Diabetes Mellitus  80380050610
4  2011-08-04                 NaN  Diabetes Mellitus  80080101704


/Users/davidhuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/davidhuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [6]:
DM2_diags['DateOfOnset'] = pd.to_datetime(DM2_diags.DateOfOnset,format='%Y-%m-%d')
DM2_diags_s = DM2_diags.sort_values(by='DateOfOnset')
DM2_diags_s.head()

,DateOfOnset,DiagnosisCode_calc,Disease,Patient_ID
28759,1980-01-01,NaN,Diabetes Mellitus,80380044622
33662,1980-01-01,NaN,Diabetes Mellitus,80380044707
38909,1980-01-01,NaN,Diabetes Mellitus,80520072587
24321,1980-01-01,NaN,Diabetes Mellitus,80620024550
36895,1980-01-01,NaN,Diabetes Mellitus,80410080363


In [7]:
# Remove duplicates 
DM2_diags_su = DM2_diags_s.drop_duplicates(['Patient_ID'])
print(len(DM2_diags_su.index)) # All DM2 patients earliest DM2 diagnosis
DM2_diags_su.head()

20934


,DateOfOnset,DiagnosisCode_calc,Disease,Patient_ID
28759,1980-01-01,NaN,Diabetes Mellitus,80380044622
33662,1980-01-01,NaN,Diabetes Mellitus,80380044707
38909,1980-01-01,NaN,Diabetes Mellitus,80520072587
24321,1980-01-01,NaN,Diabetes Mellitus,80620024550
36895,1980-01-01,NaN,Diabetes Mellitus,80410080363


### Get DM2 patients with a complication

In [8]:
## The 10 Complications
#DM2 = [float(item/100) for item in range(25000,25100)]
AP = [float(item/100) for item in range(41300,41400)] # http://www.icd9data.com/2013/Volume1/390-459/410-414/413/default.htm
AS = [float(item/100) for item in range(44000,44100)] # http://www.icd9data.com/2012/Volume1/390-459/440-449/440/default.htm
ICHD = [float(item/100) for item in range(41400,41500)] # http://www.icd9data.com/2015/Volume1/390-459/410-414/414/default.htm
DD = [311] # http://www.icd9data.com/2012/Volume1/290-319/300-316/311/default.htm
DNP = [float(item/100) for item in range(58500,58600)]
DNP.append(581.81) # include 250.4? http://www.icd9data.com/2012/Volume1/580-629/580-589/585/default.htm
DNU = [float(item/100) for item in range(35700,35800)] # include 250.6? 357.2 is the exact code http://www.icd9data.com/2012/Volume1/320-389/350-359/357/default.htm 
DR = [float(item/100) for item in range(36200,36210)] # http://www.icd9data.com/2015/Volume1/320-389/360-379/362/default.htm
HL = [389.9] #http://www.icd9data.com/2012/Volume1/320-389/380-389/389/default.htm
MI = [float(item/100) for item in range(41000,41100)] #http://www.icd9data.com/2014/Volume1/390-459/410-414/410/default.htm
PVD = [float(item/100) for item in range(43300,43400)] #http://www.icd9data.com/2013/Volume1/390-459/440-449/443/default.htm
complications = AP + AS + ICHD + DD + DNU + DNP +DR + HL + MI + PVD # + DM2

In [9]:
DM2_diags_id = DM2_diags_su['Patient_ID']
DM2_billing_set = Billing2.loc[Billing2['Patient_ID'].isin(DM2_diags_id)]
complicationDF = DM2_billing_set.loc[DM2_billing_set['DiagnosisCode_calc'].isin(complications)]
print(len(complicationDF.index))
complicationDF.head()

1826


,Billing_ID,Network_ID,Site_ID,Patient_ID,Encounter_ID,ServiceDate,ServiceCode,DiagnosisText_orig,DiagnosisText_calc,DiagnosisCodeType_orig,DiagnosisCodeType_calc,DiagnosisCode_orig,DiagnosisCode_calc,DateCreated
1459207,5789399,8,3,80034625466,8394916.0,2019-10-03,"78500, 78791",NaN,Coronary atherosclerosis of unspecified type o...,ICD9,ICD9,41400,414.0,2019-10-26
1459227,5790068,8,3,80034628749,8489973.0,2019-02-27,"25979, 78529",NaN,"DEPRESSIVE DISORDER, NOT ELSEWHERE CLASSIFIED",ICD9,ICD9,311,311.0,2019-02-27
1459236,5790077,8,3,80034628749,8273724.0,2019-05-30,78529,NaN,"DEPRESSIVE DISORDER, NOT ELSEWHERE CLASSIFIED",ICD9,ICD9,311,311.0,2019-05-30
1459237,5790078,8,3,80034628749,8411735.0,2019-06-20,78529,NaN,"DEPRESSIVE DISORDER, NOT ELSEWHERE CLASSIFIED",ICD9,ICD9,311,311.0,2019-06-20
1459255,5790216,8,3,80034629485,8253408.0,2019-04-25,78005,NaN,OTHER AND UNSPECIFIED ANGINA PECTORIS,ICD9,ICD9,4139,413.9,2019-04-25


In [10]:
# Sort and Remove Duplicates keeping latest complication diagnosis
complicationDF['ServiceDate'] = pd.to_datetime(complicationDF.ServiceDate,format='%Y-%m-%d') # Turn ServiceDate column to datetime
complicationDFS = complicationDF.sort_values(by='ServiceDate', ascending = False)
complicationDFSU = complicationDFS.drop_duplicates(subset = ['Patient_ID'])
print(len(complicationDFSU.index))
complicationDFSU.head()

776


/Users/davidhuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Billing_ID,Network_ID,Site_ID,Patient_ID,Encounter_ID,ServiceDate,ServiceCode,DiagnosisText_orig,DiagnosisText_calc,DiagnosisCodeType_orig,DiagnosisCodeType_calc,DiagnosisCode_orig,DiagnosisCode_calc,DateCreated
1651691,5946713,8,39,80394387048,10413072.0,2019-12-31,78529,NaN,"DEPRESSIVE DISORDER, NOT ELSEWHERE CLASSIFIED",ICD9,ICD9,311,311.0,2019-12-31
1583942,5885563,8,33,80333036679,10039467.0,2019-12-30,78473,NaN,"CHRONIC KIDNEY DISEASE, UNSPECIFIED",ICD9,ICD9,5859,585.9,2019-12-30
1622401,5835325,8,4,80044255650,8656736.0,2019-12-30,"78473, 78513",NaN,"CHRONIC KIDNEY DISEASE, UNSPECIFIED",ICD9,ICD9,5859,585.9,2019-12-30
1763862,5828988,8,4,80044192823,8706766.0,2019-12-30,78005,NaN,CORONARY ATHEROSCLEROSIS OF UNSPECIFIED TYPE O...,ICD9,ICD9,41400,414.0,2019-12-30
1814431,5777359,8,3,80034583058,8177627.0,2019-12-30,78005,NaN,"DEPRESSIVE DISORDER, NOT ELSEWHERE CLASSIFIED",ICD9,ICD9,311,311.0,2019-12-30


In [11]:
complication_clean = complicationDFSU[['Patient_ID', 'ServiceDate', 'DiagnosisCode_calc']]
complication_clean.columns = ['Patient_ID','CompOnset','CompCode']
complication_clean = pd.merge(complication_clean, DM2_diags_su, on='Patient_ID')
complications_final = complication_clean[['Patient_ID','DateOfOnset','CompOnset']]
complications_final.head(10)

,Patient_ID,DateOfOnset,CompOnset
0,80394387048,2011-05-09,2019-12-31
1,80333036679,2018-01-14,2019-12-30
2,80044255650,2012-12-01,2019-12-30
3,80044192823,2012-10-31,2019-12-30
4,80034583058,2016-03-09,2019-12-30
5,80333042524,2003-12-03,2019-12-30
6,80034658818,2015-07-15,2019-12-30
7,80034631904,2015-04-02,2019-12-30
8,80044340512,2018-11-09,2019-12-30
9,80394419304,2008-03-30,2019-12-27


### Collect visit data between DM2 onset and comp onset dates

In [12]:
comp_ids = complications_final['Patient_ID'].unique()
CompBillingSet = Billing2.loc[Billing2['Patient_ID'].isin(comp_ids)]
CompBillingSet['ServiceDate'] = pd.to_datetime(CompBillingSet.ServiceDate,format='%Y-%m-%d')
CompBillingSet_sorted = CompBillingSet.sort_values(by='ServiceDate')
print(len(CompBillingSet_sorted.index))
CompBillingSet_sorted.head()

10083


/Users/davidhuang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Billing_ID,Network_ID,Site_ID,Patient_ID,Encounter_ID,ServiceDate,ServiceCode,DiagnosisText_orig,DiagnosisText_calc,DiagnosisCodeType_orig,DiagnosisCodeType_calc,DiagnosisCode_orig,DiagnosisCode_calc,DateCreated
1778255,5892715,8,33,80333041324,9917359.0,2018-09-11,"78513, 89644",NaN,"ABDOMINAL PAIN, OTHER SPECIFIED SITE; MULTIPLE...",ICD9,ICD9,78909,789.09,2019-03-05
1469140,5730312,8,1,80013281045,NaN,2018-09-21,78513,NaN,CORONARY ATHEROSCLEROSIS OF UNSPECIFIED TYPE O...,ICD9,ICD9,41400,414.00,2019-03-04
1756210,5676814,8,1,80013138602,7119087.0,2018-10-17,78513,NaN,UNSPECIFIED INFLAMMATORY AND TOXIC NEUROPATHIES,ICD9,ICD9,3579,357.90,2019-03-06
1808756,5705450,8,1,80013234302,NaN,2018-10-21,78520,NaN,"CONGESTIVE HEART FAILURE, UNSPECIFIED",ICD9,ICD9,4280,428.00,2019-02-08
1778258,5892718,8,33,80333041324,10032185.0,2018-10-30,78513,NaN,SHORTNESS OF BREATH,ICD9,ICD9,78605,786.05,2019-04-16


In [13]:
# Combine dfs
CompBillingSet_merged = pd.merge(CompBillingSet_sorted, complications_final, on='Patient_ID')

In [17]:
# Gather all patient visits per patient between diagnosis' in a dictionary
AllVisits = {}
for index, row in CompBillingSet_merged.iterrows() :
    temp = (str(row['Patient_ID']),str(row['ServiceDate']))
    dateVisit = row['ServiceDate']
    dateOnset = row['DateOfOnset']
    dateComp = row['CompOnset']
    diag = row['DiagnosisCode_calc']
    if temp not in AllVisits.keys():
        AllVisits[temp] = []
    if dateOnset <= dateVisit <= dateComp :
        AllVisits[temp] = AllVisits[temp] + [diag]

In [18]:
# Turn dictionary to dataframe
test = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AllVisits.items() ]))
test = test.transpose()
test = test.dropna(axis=0,how='all') # Remove NA
print(len(test.index))
test.head(30)

5910


0       1   2   3   4   5
80333041324 2018-09-11 00:00:00  789.09     NaN NaN NaN NaN NaN
            2018-10-30 00:00:00  786.05     NaN NaN NaN NaN NaN
            2018-11-05 00:00:00  511.90     NaN NaN NaN NaN NaN
            2018-12-04 00:00:00  797.00     NaN NaN NaN NaN NaN
            2019-02-26 00:00:00  780.52     NaN NaN NaN NaN NaN
            2019-04-04 00:00:00  401.90     NaN NaN NaN NaN NaN
            2019-04-05 00:00:00  780.52     NaN NaN NaN NaN NaN
            2019-04-16 00:00:00  311.00     NaN NaN NaN NaN NaN
            2019-04-25 00:00:00  797.00     NaN NaN NaN NaN NaN
            2019-09-04 00:00:00  300.02     NaN NaN NaN NaN NaN
            2019-09-18 00:00:00  311.00     NaN NaN NaN NaN NaN
80013281045 2019-04-04 00:00:00  401.90     NaN NaN NaN NaN NaN
            2019-04-26 00:00:00  414.00     NaN NaN NaN NaN NaN
            2019-06-12 00:00:00  414.00  250.02 NaN NaN NaN NaN
            2019-08-26 00:00:00  250.02  414.00 NaN NaN NaN NaN
            2019-09-18 00:00:00  133.00     NaN NaN NaN NaN NaN
            2019-10-09 00:00:00  414.00     NaN NaN NaN NaN NaN
            2019-12-06 00:00:00  414.00     NaN NaN NaN NaN NaN
80013138602 2018-10-17 00:00:00  357.90     NaN NaN NaN NaN NaN
80013234302 2018-10-21 00:00:00  428.00     NaN NaN NaN NaN NaN
            2019-03-05 00:00:00  428.00     NaN NaN NaN NaN NaN
            2019-03-12 00:00:00  428.00     NaN NaN NaN NaN NaN
            2019-03-19 00:00:00  428.00     NaN NaN NaN NaN NaN
            2019-03-26 00:00:00  250.00     NaN NaN NaN NaN NaN
            2019-04-01 00:00:00  250.00     NaN NaN NaN NaN NaN
            2019-04-02 00:00:00  250.00     NaN NaN NaN NaN NaN
            2019-04-03 00:00:00  428.00     NaN NaN NaN NaN NaN
            2019-04-04 00:00:00  414.00     NaN NaN NaN NaN NaN
80064089752 2018-10-31 00:00:00  493.90     NaN NaN NaN NaN NaN
            2018-11-28 00:00:00  311.00     NaN NaN NaN NaN NaN

### Feature vector creation

In [19]:
# Check # of unique ICD9 codes
len(Billing2['DiagnosisCode_calc'].value_counts().index)

1444

In [20]:
# Remove ICD9 codes that only appear once
ICD9s = Billing2['DiagnosisCode_calc'].value_counts()
ICD9s = ICD9s[Billing2['DiagnosisCode_calc'].value_counts() > 1]
ICD9s.index

Float64Index([ 401.9, 250.02,  250.0,  300.0,  311.0, 286.59,  401.1,  414.0,
              530.11, 250.93,
              ...
               297.1, 820.01,  626.7, 719.09,  915.4,  923.8,  79.98,  721.3,
               460.0,  625.4],
             dtype='float64', length=1209)

In [21]:
# Create new array for SVD input where columns = all possible ICD9 codes, rows are each visit, filled in with 0s
RowFeatures = pd.DataFrame(data = 0,
                           index = test.index,
                           columns=ICD9s.index)
RowFeatures.head()

401.90  250.02  250.00  300.00  311.00  \
80333041324 2018-09-11 00:00:00       0       0       0       0       0   
            2018-10-30 00:00:00       0       0       0       0       0   
            2018-11-05 00:00:00       0       0       0       0       0   
            2018-12-04 00:00:00       0       0       0       0       0   
            2019-02-26 00:00:00       0       0       0       0       0   

                                 286.59  401.10  414.00  530.11  250.93  ...  \
80333041324 2018-09-11 00:00:00       0       0       0       0       0  ...   
            2018-10-30 00:00:00       0       0       0       0       0  ...   
            2018-11-05 00:00:00       0       0       0       0       0  ...   
            2018-12-04 00:00:00       0       0       0       0       0  ...   
            2019-02-26 00:00:00       0       0       0       0       0  ...   

                                 297.10  820.01  626.70  719.09  915.40  \
80333041324 2018-09-11 00:00:00       0       0       0       0       0   
            2018-10-30 00:00:00       0       0       0       0       0   
            2018-11-05 00:00:00       0       0       0       0       0   
            2018-12-04 00:00:00       0       0       0       0       0   
            2019-02-26 00:00:00       0       0       0       0       0   

                                 923.80  79.98   721.30  460.00  625.40  
80333041324 2018-09-11 00:00:00       0       0       0       0       0  
            2018-10-30 00:00:00       0       0       0       0       0  
            2018-11-05 00:00:00       0       0       0       0       0  
            2018-12-04 00:00:00       0       0       0       0       0  
            2019-02-26 00:00:00       0       0       0       0       0  

[5 rows x 1209 columns]

In [22]:
# if diag occured in a visit, col[diagnosisCode] = 1
for index, row in RowFeatures.iterrows() :
    for diag in test.loc[index].dropna() :
        row[diag] = 1
    

In [23]:
# Take a look at final matrix
RowFeatures

401.90  250.02  250.00  300.00  311.00  \
80333041324 2018-09-11 00:00:00       0       0       0       0       0   
            2018-10-30 00:00:00       0       0       0       0       0   
            2018-11-05 00:00:00       0       0       0       0       0   
            2018-12-04 00:00:00       0       0       0       0       0   
            2019-02-26 00:00:00       0       0       0       0       0   
...                                 ...     ...     ...     ...     ...   
80613318971 2019-12-02 00:00:00       0       0       0       0       1   
80333044822 2019-10-21 00:00:00       0       0       0       0       1   
80034908119 2019-10-22 00:00:00       0       0       0       0       0   
80394385713 2019-11-12 00:00:00       0       0       0       0       1   
80333038918 2019-12-23 00:00:00       0       0       0       0       0   

                                 286.59  401.10  414.00  530.11  250.93  ...  \
80333041324 2018-09-11 00:00:00       0       0       0       0       0  ...   
            2018-10-30 00:00:00       0       0       0       0       0  ...   
            2018-11-05 00:00:00       0       0       0       0       0  ...   
            2018-12-04 00:00:00       0       0       0       0       0  ...   
            2019-02-26 00:00:00       0       0       0       0       0  ...   
...                                 ...     ...     ...     ...     ...  ...   
80613318971 2019-12-02 00:00:00       0       0       0       0       0  ...   
80333044822 2019-10-21 00:00:00       0       0       0       0       0  ...   
80034908119 2019-10-22 00:00:00       0       0       1       0       1  ...   
80394385713 2019-11-12 00:00:00       0       0       0       0       0  ...   
80333038918 2019-12-23 00:00:00       0       0       0       0       0  ...   

                                 297.10  820.01  626.70  719.09  915.40  \
80333041324 2018-09-11 00:00:00       0       0       0       0       0   
            2018-10-30 00:00:00       0       0       0       0       0   
            2018-11-05 00:00:00       0       0       0       0       0   
            2018-12-04 00:00:00       0       0       0       0       0   
            2019-02-26 00:00:00       0       0       0       0       0   
...                                 ...     ...     ...     ...     ...   
80613318971 2019-12-02 00:00:00       0       0       0       0       0   
80333044822 2019-10-21 00:00:00       0       0       0       0       0   
80034908119 2019-10-22 00:00:00       0       0       0       0       0   
80394385713 2019-11-12 00:00:00       0       0       0       0       0   
80333038918 2019-12-23 00:00:00       0       0       0       0       0   

                                 923.80  79.98   721.30  460.00  625.40  
80333041324 2018-09-11 00:00:00       0       0       0       0       0  
            2018-10-30 00:00:00       0       0       0       0       0  
            2018-11-05 00:00:00       0       0       0       0       0  
            2018-12-04 00:00:00       0       0       0       0       0  
            2019-02-26 00:00:00       0       0       0       0       0  
...                                 ...     ...     ...     ...     ...  
80613318971 2019-12-02 00:00:00       0       0       0       0       0  
80333044822 2019-10-21 00:00:00       0       0       0       0       0  
80034908119 2019-10-22 00:00:00       0       0       0       0       0  
80394385713 2019-11-12 00:00:00       0       0       0       0       0  
80333038918 2019-12-23 00:00:00       0       0       0       0       0  

[5910 rows x 1209 columns]

### SVD 

In [27]:
# SVD matrices
U, s, V = np.linalg.svd(RowFeatures)
print(U.shape, V.shape, s.shape)

In [28]:
S = np.zeros((RowFeatures.shape[0], RowFeatures.shape[1]))
S[:RowFeatures.shape[1], :RowFeatures.shape[1]] = np.diag(s)
n_component = 50
S = S[:, :n_component]

In [31]:
reducedMat = U.dot(S)

In [32]:
reducedMat.shape

(5910, 50)

In [35]:
reducedDF = pd.DataFrame(data=reducedMat, index=RowFeatures.index)
reducedDF

0         1         2         3   \
80333041324 2018-09-11 00:00:00 -0.000061 -0.000155  0.001436  0.000225   
            2018-10-30 00:00:00 -0.000231 -0.001314 -0.000202  0.000088   
            2018-11-05 00:00:00 -0.000243 -0.000625  0.005779  0.000912   
            2018-12-04 00:00:00 -0.000962  0.000331 -0.000053  0.000021   
            2019-02-26 00:00:00 -0.002191  0.000366  0.001466  0.000594   
...                                   ...       ...       ...       ...   
80613318971 2019-12-02 00:00:00 -0.061856 -0.116085  0.986017  0.085171   
80333044822 2019-10-21 00:00:00 -0.061856 -0.116085  0.986017  0.085171   
80034908119 2019-10-22 00:00:00 -0.272705 -1.060597 -0.122636  0.064374   
80394385713 2019-11-12 00:00:00 -0.061856 -0.116085  0.986017  0.085171   
80333038918 2019-12-23 00:00:00 -0.003056  0.000765 -0.000100  0.000442   

                                       4         5         6             7   \
80333041324 2018-09-11 00:00:00 -0.000074  0.000175  0.000099  4.313052e-07   
            2018-10-30 00:00:00 -0.000183  0.000696 -0.000092  7.275925e-06   
            2018-11-05 00:00:00 -0.000303  0.000626  0.000410  1.532179e-06   
            2018-12-04 00:00:00 -0.000177  0.000134  0.000021 -1.234617e-05   
            2019-02-26 00:00:00  0.006605 -0.000007  0.000246 -1.022930e-04   
...                                   ...       ...       ...           ...   
80613318971 2019-12-02 00:00:00 -0.024533  0.036861  0.012094  4.716218e-05   
80333044822 2019-10-21 00:00:00 -0.024533  0.036861  0.012094  4.716218e-05   
80034908119 2019-10-22 00:00:00 -0.109988 -0.843381  0.001857 -2.181956e-03   
80394385713 2019-11-12 00:00:00 -0.024533  0.036861  0.012094  4.716218e-05   
80333038918 2019-12-23 00:00:00  0.008564 -0.000244  0.000083 -9.270156e-05   

                                       8         9   ...        40        41  \
80333041324 2018-09-11 00:00:00 -0.000005 -0.000118  ...  0.000026  0.000003   
            2018-10-30 00:00:00 -0.000352  0.001578  ...  0.000126 -0.001031   
            2018-11-05 00:00:00 -0.000024 -0.000524  ...  0.000230  0.000546   
            2018-12-04 00:00:00  0.000008 -0.000033  ... -0.001710 -0.000030   
            2019-02-26 00:00:00  0.000921 -0.000016  ...  0.001263 -0.028683   
...                                   ...       ...  ...       ...       ...   
80613318971 2019-12-02 00:00:00 -0.000535 -0.010917  ...  0.000369  0.000038   
80333044822 2019-10-21 00:00:00 -0.000535 -0.010917  ...  0.000369  0.000038   
80034908119 2019-10-22 00:00:00  0.000427 -0.092956  ...  0.007363  0.004192   
80394385713 2019-11-12 00:00:00 -0.000535 -0.010917  ...  0.000369  0.000038   
80333038918 2019-12-23 00:00:00  0.000859  0.000273  ... -0.001889 -0.002016   

                                       42        43        44        45  \
80333041324 2018-09-11 00:00:00  0.000650 -0.000009  0.000213  0.000456   
            2018-10-30 00:00:00  0.010711  0.001473  0.000055  0.010689   
            2018-11-05 00:00:00  0.005466  0.000109 -0.004600  0.004823   
            2018-12-04 00:00:00  0.000197 -0.000028  0.000213 -0.000144   
            2019-02-26 00:00:00 -0.003460 -0.000839  0.026213 -0.003308   
...                                   ...       ...       ...       ...   
80613318971 2019-12-02 00:00:00  0.007166 -0.000087  0.002091  0.004144   
80333044822 2019-10-21 00:00:00  0.007166 -0.000087  0.002091  0.004144   
80034908119 2019-10-22 00:00:00  0.008890 -0.000118  0.002193  0.001806   
80394385713 2019-11-12 00:00:00  0.007166 -0.000087  0.002091  0.004144   
80333038918 2019-12-23 00:00:00  0.000995 -0.001453  0.001617  0.001718   

                                       46        47        48        49  
80333041324 2018-09-11 00:00:00  0.000142  0.000927  0.000189  0.000765  
            2018-10-30 00:00:00  0.001524  0.006937 -0.000893 -0.001303  
            2018-11-05 00:00:00  0.001491  0.009792  0.003024  0.010491  
            2018-12-04 00:00:00

In [68]:
# Create dictionary with patients as keys and columns as SVD vectors
VisitHistorySVD = {}
for index, row in reducedDF.iterrows() :
    temp = str(index[0])
    featureVector = np.array(row)
    if temp not in VisitHistorySVD.keys():
        VisitHistorySVD[temp] = []
    VisitHistorySVD[temp] = VisitHistorySVD[temp] + [featureVector]

In [95]:
# Convert dictionary to dataframe
VisitHistoryDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in VisitHistorySVD.items() ]))
VisitHistoryDF = VisitHistoryDF.transpose()
VisitHistoryDF

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
80333041324,"[-6.053848872998231e-05, -0.000155085323421573...","[-0.00023080704296427813, -0.00131351052281816...","[-0.0002433561460540403, -0.000624678679104822...","[-0.0009617589070007073, 0.0003306644358279100...","[-0.0021909748994215516, 0.0003662121746413517...","[-0.9672943326212354, 0.2422188044732389, -0.0...","[-0.0021909748994217606, 0.0003662121746403473...","[-0.06185553522064403, -0.1160846970123338, 0....","[-0.0009617589070007278, 0.0003306644358280804...","[-0.0014397151083415242, -0.000927138167075956...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80013281045,"[-0.9672943326212349, 0.24221880447323915, -0....","[-0.2247062309276137, -0.9509056237728316, -0....","[-0.29519451951834663, -1.0009532341618899, -0...","[-0.29519451951834647, -1.0009532341618896, -0...","[1.121831290899406e-18, 7.25366258807678e-19, ...","[-0.2247062309276138, -0.9509056237728304, -0....","[-0.22470623092761366, -0.9509056237728287, -0...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80013138602,"[-4.562324629630962e-06, -1.4982361029940576e-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80013234302,"[-0.009371221197963433, -0.022713913182921954,...","[-0.009371221197963417, -0.022713913182921933,...","[-0.009371221197963417, -0.022713913182921923,...","[-0.009371221197963403, -0.022713913182921916,...","[-0.026785181654436926, -0.054581276560878134,...","[-0.026785181654436947, -0.05458127656087811, ...","[-0.026785181654436947, -0.05458127656087808, ...","[-0.009371221197963414, -0.02271391318292193, ...","[-0.22470623092761374, -0.9509056237728303, -0...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80064089752,"[-0.00036495475704526037, -0.00168155779525150...","[-0.061855535220644756, -0.11608469701233678, ...","[-0.06185553522064476, -0.1160846970123369, 0....","[1.5840400241779351e-18, 1.0132346019701723e-1...","[2.4478782821884863e-19, 7.540710793509466e-19...","[-1.443591911731953e-18, -3.397921741007625e-1...","[2.319811709734904e-19, 4.807829624961443e-19,...","[6.616950042290466e-19, 1.3982437029076647e-18...","[-0.06185553522064475, -0.11608469701233683, 0...","[-0.06185553522064477, -0.11608469701233681, 0...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80613318971,"[-7.099319506107239e-05, -7.501768634205412e-0...","[-1.3691099937831097e-23, 3.435467759037921e-2...","[-0.06185553522064477, -0.11608469701233669, 0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80333044822,"[-0.06185553522064477, -0.11608469701233669, 0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80034908119,"[-0.27270527433666697, -1.0605966937561722, -0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80394385713,"[-0.06185553522064477, -0.11608469701233669, 0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
# Convert NaN cells to np.zeros(50)
for i in range(0,VisitHistoryDF.shape[0]) :
    for j in range(0,VisitHistoryDF.shape[1]) :
        if type(VisitHistoryDF.iloc[i][j]) != type(VisitHistoryDF.iloc[0][0]):
            VisitHistoryDF.iloc[i][j] = np.zeros(50)

In [108]:
VisitHistoryDF

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
80333041324,"[-6.053848872998231e-05, -0.000155085323421573...","[-0.00023080704296427813, -0.00131351052281816...","[-0.0002433561460540403, -0.000624678679104822...","[-0.0009617589070007073, 0.0003306644358279100...","[-0.0021909748994215516, 0.0003662121746413517...","[-0.9672943326212354, 0.2422188044732389, -0.0...","[-0.0021909748994217606, 0.0003662121746403473...","[-0.06185553522064403, -0.1160846970123338, 0....","[-0.0009617589070007278, 0.0003306644358280804...","[-0.0014397151083415242, -0.000927138167075956...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
80013281045,"[-0.9672943326212349, 0.24221880447323915, -0....","[-0.2247062309276137, -0.9509056237728316, -0....","[-0.29519451951834663, -1.0009532341618899, -0...","[-0.29519451951834647, -1.0009532341618896, -0...","[1.121831290899406e-18, 7.25366258807678e-19, ...","[-0.2247062309276138, -0.9509056237728304, -0....","[-0.22470623092761366, -0.9509056237728287, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
80013138602,"[-4.562324629630962e-06, -1.4982361029940576e-...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
80013234302,"[-0.009371221197963433, -0.022713913182921954,...","[-0.009371221197963417, -0.022713913182921933,...","[-0.009371221197963417, -0.022713913182921923,...","[-0.009371221197963403, -0.022713913182921916,...","[-0.026785181654436926, -0.054581276560878134,...","[-0.026785181654436947, -0.05458127656087811, ...","[-0.026785181654436947, -0.05458127656087808, ...","[-0.009371221197963414, -0.02271391318292193, ...","[-0.22470623092761374, -0.9509056237728303, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [ ]:
# TODO: 
# remove patients with only one diagnosis (visit)
# Create negative samples